<a href="https://colab.research.google.com/github/skillox1/PADSW06MDCD/blob/main/Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Base de Dados Olist

In [25]:
import pandas as pd
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt

Questões 1 a 8

In [27]:
zf = zipfile.ZipFile('/content/archive.zip') 
df_orders = pd.read_csv(zf.open('olist_orders_dataset.csv'))
df_order_items = pd.read_csv(zf.open('olist_order_items_dataset.csv'))
df_customers = pd.read_csv(zf.open('olist_customers_dataset.csv'))
df_sellers = pd.read_csv(zf.open('olist_sellers_dataset.csv'))
df_products = pd.read_csv(zf.open('olist_products_dataset.csv'))
df_order_payments = pd.read_csv(zf.open('olist_order_payments_dataset.csv'))
df_order_reviews = pd.read_csv(zf.open('olist_order_reviews_dataset.csv'))
df_geolocation = pd.read_csv(zf.open('olist_geolocation_dataset.csv'))



In [29]:
dfs = [df_orders,df_order_items,df_customers,df_sellers,df_products,df_order_payments,df_order_reviews,df_geolocation]

In [ ]:
for x in dfs:
  print(x.shape[0])

99441
112650
99441
3095
32951
103886
99224
1000163


Questão 9

In [30]:
df_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [31]:
df_geolocation.geolocation_zip_code_prefix.nunique()

19015

Questão 10

In [32]:
df_geolocation.geolocation_lat.value_counts().sum()

1000163

In [33]:
df_geolocation.geolocation_lng.value_counts().sum()

1000163

Questão 11

In [34]:
df_geolocation.groupby("geolocation_zip_code_prefix").head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
999961,99960,-27.953858,-52.031943,charrua,RS
999996,99952,-28.182027,-51.880215,santa cecilia do sul,RS
1000008,99925,-27.936330,-52.425730,ipiranga do sul,RS
1000067,99965,-28.174697,-52.035508,agua santa,RS


In [48]:
df_geolocation.groupby("geolocation_zip_code_prefix").agg({"geolocation_lat":["mean"],"geolocation_lng":["mean"],"geolocation_city":[pd.Series.mode],\
                                                           "geolocation_state":[pd.Series.mode]})

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
,mean,mean,mode,mode
geolocation_zip_code_prefix,,,,
1001,-23.550190,-46.634024,sao paulo,SP
1002,-23.548146,-46.634979,sao paulo,SP
1003,-23.548994,-46.635731,sao paulo,SP
1004,-23.549799,-46.634757,sao paulo,SP
1005,-23.549456,-46.636733,sao paulo,SP
...,...,...,...,...
99960,-27.953722,-52.025511,charrua,RS
99965,-28.183372,-52.039850,agua santa,RS


In [49]:
df_geolocation_adj_coord = df_geolocation.groupby("geolocation_zip_code_prefix").agg({"geolocation_lat":["mean"],"geolocation_lng":["mean"],"geolocation_city":[pd.Series.mode],\
                                                           "geolocation_state":[pd.Series.mode]})

In [50]:
df_customers.rename(columns={"customer_zip_code_prefix": "zip_code"}, inplace=True)
df_geolocation_adj_coord.rename(columns={"geolocation_zip_code_prefix": "zip_code"}, inplace=True)

In [ ]:
data = orders_df.merge(customers_df, on="customer_id").merge(orderitem_df, on="order_id").merge(products_df, on="product_id").merge(categname_df, on="product_category_name")\
      .merge(orderpay_df, on="order_id").merge(sellers_df, on="seller_id").merge(orderreviews_df, on="order_id")